<a href="https://colab.research.google.com/github/JK-the-Ko/Thermo-Fluid-Dynamics-Experiment/blob/main/2023-2/%EC%97%B4%EC%9C%A0%EC%B2%B4%EA%B3%B5%ED%95%99%EC%8B%A4%ED%97%981_Week11_PyTorch_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Classification Using PyTorch Framework

## Check NVIDIA GPU Setting

In [ ]:
!nvidia-smi

## Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Unzip Dataset

In [ ]:
!unzip "/content/drive/MyDrive/dataset.zip" # path to dataset

## Get Number of Data

In [ ]:
from os import listdir
from os.path import join

In [ ]:
sourcePath = "/content/" # path to dataset

In [ ]:
print("<Training Dataset>")
for className in listdir(join(sourcePath, "train")) :
  print(f"{className}:{len(listdir(join(sourcePath, 'train', className)))}")

In [ ]:
print("<Test Dataset>")
for className in listdir(join(sourcePath, "test")) :
  print(f"{className}:{len(listdir(join(sourcePath, 'test', className)))}")

## Create PyTorch DataLoader Class without Data Augmentation

In [ ]:
from PIL import Image

import torch
from torch.utils.data import Dataset
from torchvision import transforms

In [ ]:
class myDataLoader(Dataset) :
  def __init__(self, opt, forMetric) :
    super(myDataLoader, self).__init__()

    self.opt = opt
    self.forMetric = forMetric
    self.imageDataset = self.getPathList()
    self.label = {"cloudy":[1,0,0,0],
                  "desert":[0,1,0,0],
                  "green_area":[0,0,1,0],
                  "water":[0,0,0,1]} # One-Hot Encoding

  def __getitem__(self, index) :
    image = Image.open(self.imageDataset[0][index]).convert("RGB") #4D to 3D
    image = self.transforms(image)

    label = self.label[self.imageDataset[1][index]]

    return {"image":image, "label":torch.as_tensor(label).float()}

  def __len__(self) :
    return len(self.imageDataset[1])

  def getPathList(self) :
    if self.forMetric :
      classPath = join(self.opt["dataRoot"], "test")
    else :
      classPath = join(self.opt["dataRoot"], "train")

    imagePathList, imageLabelList = [], []
    for className in listdir(classPath) :
      for imageName in listdir(join(classPath, className)) :
        imagePathList.append(join(classPath, className, imageName))
        imageLabelList.append(className)

    return (imagePathList, imageLabelList)

  def transforms(self, image) :
    if self.forMetric :
      myTransforms = transforms.Compose([transforms.Resize(self.opt["cropSize"]),
                                        transforms.ToTensor()]) # Resize Process for Test
    else :
      myTransforms = transforms.Compose([transforms.RandomCrop(self.opt["cropSize"]),
                                        transforms.ToTensor()]) # Random Crop for Training
    image = myTransforms(image)

    return image

## Create PyTorch Image Classification Model

In [ ]:
from torch import nn
import torch.nn.functional as F

In [ ]:
class myModel(nn.Module) :
  def __init__(self, opt) :
    super(myModel, self).__init__()

    inputDim, targetDim, channels = opt["inputDim"], opt["targetDim"], opt["channels"]

    self.layer0 = nn.Sequential(nn.Conv2d(inputDim, channels, kernel_size=3, stride=1, padding=1),
                                nn.ReLU(),
                                nn.MaxPool2d(kernel_size=2, stride=2))
    self.layer1 = nn.Sequential(nn.Conv2d(channels, channels*2, kernel_size=3, stride=1, padding=1),
                                nn.ReLU(),
                                nn.MaxPool2d(kernel_size=2, stride=2))
    self.layer2 = nn.Sequential(nn.Conv2d(channels*2, channels*4, kernel_size=3, stride=1, padding=1),
                                nn.ReLU(),
                                nn.MaxPool2d(kernel_size=2, stride=2))
    self.layer3 = nn.Sequential(nn.Conv2d(channels*4, channels*4, kernel_size=3, stride=1, padding=1),
                                nn.ReLU(),
                                nn.MaxPool2d(kernel_size=2, stride=2))
    self.layer4 = nn.Sequential(nn.Conv2d(channels*4, channels*4, kernel_size=3, stride=1, padding=1),
                                nn.ReLU(),
                                nn.MaxPool2d(kernel_size=2, stride=2))
    self.layer5 = nn.Sequential(nn.Conv2d(channels*4, channels*4, kernel_size=3, stride=1, padding=1),
                                nn.ReLU(),
                                nn.MaxPool2d(kernel_size=2, stride=2))
    self.layer6 = nn.Linear(channels*4, targetDim)

  def forward(self, input) :
    output = self.layer0(input)
    output = self.layer1(output)
    output = self.layer2(output)
    output = self.layer3(output)
    output = self.layer4(output)
    output = self.layer5(output)
    output = F.adaptive_avg_pool2d(output, (1,1)).view(output.size(0), -1)
    output = self.layer6(output)

    return output

## Train DL Model

In [ ]:
from torch.utils.data import DataLoader
from torch import optim

from torchsummary import summary

from tqdm import tqdm

### Fix Seed

In [ ]:
import random
import numpy as np

In [ ]:
def fixSeed(seed) :
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

## Create Average Meter Instance

In [ ]:
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val*n
        self.count += n
        self.avg = self.sum / self.count

## Create Accuracy Computation Function

In [ ]:
def computeAcc(pred, target) :
  acc = (torch.argmax(pred, dim=1)==torch.argmax(target, dim=1)).sum()/pred.size(0)

  return acc

## Training Code as a Function (Abstraction)

In [ ]:
def train(opt, myDataLoader, myModel, criterion) :
  fixSeed(opt["seed"])

  trainDataLoader = DataLoader(myDataLoader(opt, forMetric=False), batch_size=opt["batchSize"], shuffle=True, drop_last=True)
  testDataLoader = DataLoader(myDataLoader(opt, forMetric=True), batch_size=opt["batchSize"], shuffle=False, drop_last=False)

  fixSeed(opt["seed"])
  model = myModel(opt)
  if opt["isCUDA"] :
    model = model.cuda()

  summary(model, (opt["inputDim"], opt["cropSize"], opt["cropSize"]))

  optimizer = optim.Adam(model.parameters(), lr=opt["lr"])

  trainLoss, testLoss = AverageMeter(), AverageMeter()
  trainAcc, testAcc = AverageMeter(), AverageMeter()
  trainLossList, testLossList = [], []
  trainAccList, testAccList = [], []
  bestAcc = 0

  for epoch in range(1, opt["epochs"]+1) :
    trainBar = tqdm(trainDataLoader)
    trainLoss.reset(), trainAcc.reset()

    for data in trainBar :
      input, target = data["image"], data["label"]
      if opt["isCUDA"] :
        input, target = input.cuda(), target.cuda()

      optimizer.zero_grad()
      pred = model(input)
      loss = criterion(pred, target)
      loss.backward()
      optimizer.step()

      trainLoss.update(loss.item(), opt["batchSize"])
      trainAcc.update(computeAcc(pred, target).item(), opt["batchSize"])
      trainBar.set_description(desc=f"[{epoch}/{opt['epochs']}] [Train] < Accuracy:{trainAcc.avg:.6f} | Loss:{trainLoss.avg:.6f} >")

    trainLossList.append(trainLoss.avg)
    trainAccList.append(trainAcc.avg)

    testBar = tqdm(testDataLoader)
    testLoss.reset(), testAcc.reset()

    for data in testBar :
      input, target = data["image"], data["label"]
      if opt["isCUDA"] :
        input, target = input.cuda(), target.cuda()

      model.eval()
      with torch.no_grad() :
        pred = model(input)
        loss = criterion(pred, target)

        testLoss.update(loss.item(), opt["batchSize"])
        testAcc.update(computeAcc(pred, target).item(), opt["batchSize"])
        testBar.set_description(desc=f"[{epoch}/{opt['epochs']}] [Test] < Accuracy:{testAcc.avg:.6f} | Loss:{testLoss.avg:.6f} >")

    testLossList.append(testLoss.avg)
    testAccList.append(testAcc.avg)

    if testAcc.avg > bestAcc :
      bestAcc = testAcc.avg
      torch.save(model.state_dict(), "bestModel.pth")

    torch.save(model.state_dict(), "latestModel.pth")

  return (trainLossList, testLossList), (trainAccList, testAccList)

## Create Training Option (Hyperparameter) Dictionary

In [ ]:
opt = {"dataRoot":"/content/",
       "cropSize":224,
       "seed":42,
       "inputDim":3,
       "targetDim":4,
       "channels":64,
       "batchSize":16,
       "lr":1e-4,
       "epochs":10,
       "isCUDA":torch.cuda.is_available()}

## Train Model

In [ ]:
lossList, accList = train(opt, myDataLoader, myModel, nn.CrossEntropyLoss())

## Plot Training vs. Validation Loss Graph

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(20,10))

plt.plot(np.arange(0, opt["epochs"], 1), lossList[0], label="Training Loss")
plt.plot(np.arange(0, opt["epochs"], 1), lossList[1], label="Test Loss")

plt.xlabel("Epoch")
plt.ylabel("CE Loss")
plt.legend(loc="best")

plt.show()

## Plot Training vs. Validation Accuracy Graph

In [ ]:
plt.figure(figsize=(20,10))

plt.plot(np.arange(0, opt["epochs"], 1), accList[0], label="Training Accuracy")
plt.plot(np.arange(0, opt["epochs"], 1), accList[1], label="Validation Accuracy")

plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(loc="best")

plt.show()

## Image Classification Model with Dropout and BN

In [ ]:
class myModel(nn.Module) :
  def __init__(self, opt) :
    super(myModel, self).__init__()

    inputDim, targetDim, channels = opt["inputDim"], opt["targetDim"], opt["channels"]

    self.layer0 = nn.Sequential(nn.Conv2d(inputDim, channels, kernel_size=3, stride=1, padding=1, bias=False),
                                nn.BatchNorm2d(channels),
                                nn.ReLU(),
                                nn.MaxPool2d(kernel_size=2, stride=2))
    self.layer1 = nn.Sequential(nn.Conv2d(channels, channels*2, kernel_size=3, stride=1, padding=1, bias=False),
                                nn.BatchNorm2d(channels*2),
                                nn.ReLU(),
                                nn.MaxPool2d(kernel_size=2, stride=2))
    self.layer2 = nn.Sequential(nn.Conv2d(channels*2, channels*4, kernel_size=3, stride=1, padding=1, bias=False),
                                nn.BatchNorm2d(channels*4),
                                nn.ReLU(),
                                nn.MaxPool2d(kernel_size=2, stride=2))
    self.layer3 = nn.Sequential(nn.Conv2d(channels*4, channels*4, kernel_size=3, stride=1, padding=1, bias=False),
                                nn.BatchNorm2d(channels*4),
                                nn.ReLU(),
                                nn.MaxPool2d(kernel_size=2, stride=2))
    self.layer4 = nn.Sequential(nn.Conv2d(channels*4, channels*4, kernel_size=3, stride=1, padding=1, bias=False),
                                nn.BatchNorm2d(channels*4),
                                nn.ReLU(),
                                nn.MaxPool2d(kernel_size=2, stride=2))
    self.layer5 = nn.Sequential(nn.Conv2d(channels*4, channels*4, kernel_size=3, stride=1, padding=1, bias=False),
                                nn.BatchNorm2d(channels*4),
                                nn.ReLU(),
                                nn.MaxPool2d(kernel_size=2, stride=2))
    self.layer6 = nn.Sequential(nn.Dropout(),
                                nn.Linear(channels*4, targetDim))

  def forward(self, input) :
    output = self.layer0(input)
    output = self.layer1(output)
    output = self.layer2(output)
    output = self.layer3(output)
    output = self.layer4(output)
    output = self.layer5(output)
    output = F.adaptive_avg_pool2d(output, (1,1)).view(output.size(0), -1)
    output = self.layer6(output)

    return output

In [ ]:
regLossList, regAccList = train(opt, myDataLoader, myModel, nn.CrossEntropyLoss())

## Vanilla Network vs. Network with Dropout & BN

### Loss Graph

In [ ]:
plt.figure(figsize=(20,10))

plt.plot(np.arange(0, opt["epochs"], 1), lossList[1], label="Vanilla Model Loss")
plt.plot(np.arange(0, opt["epochs"], 1), regLossList[1], label="Model with Dropout & BN Loss")

plt.xlabel("Epoch")
plt.ylabel("CE Loss")
plt.legend(loc="best")

plt.show()

### Accuracy Graph

In [ ]:
plt.figure(figsize=(20,10))

plt.plot(np.arange(0, opt["epochs"], 1), accList[1], label="Vanilla Model Loss")
plt.plot(np.arange(0, opt["epochs"], 1), regAccList[1], label="Model with Dropout & BN Loss")

plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(loc="best")

plt.show()